Import Libraries

In [ ]:
import boto3
import sagemaker
import joblib
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import pandas as pd
import numpy as np
region = boto3.Session().region_name
sess = sagemaker.Session()
from io import BytesIO
import lightgbm as lgb 

In [4]:
def read_joblib(path):
    ''' 
       Function to load a joblib file from an s3 bucket or local directory.
       Arguments:
       * path: an s3 bucket or local directory path where the file is stored
       Outputs:
       * file: Joblib file loaded
    '''

    # Path is an s3 bucket
    if path[:5] == 's3://':
        s3_bucket, s3_key = path.split('/')[2], path.split('/')[3:]
        s3_key = '/'.join(s3_key)
        with BytesIO() as f:
            boto3.client("s3").download_fileobj(Bucket=s3_bucket, Key=s3_key, Fileobj=f)
            f.seek(0)
            file = joblib.load(f)
    
    # Path is a local directory 
    else:
        with open(path, 'rb') as f:
            file = joblib.load(f)
    
    return file

Read the trained model from S3

In [5]:
mdl_lightgbm = read_joblib('s3://msba6330trendsmarketplace/model-save/mdl_dict.joblib')
mdl_lightgbm = mdl_lightgbm['mdl_fitted']

In [6]:
mdl_lightgbm

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', is_unbalance=True, learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

Read stored data from S3 and merge it with other customer files

In [7]:
## Test data 
bucket = 's3://msba6330trendsmarketplace'
# file names
test = 'application_test.csv'
#train = 'application_train.csv'
bureau_bal = 'bureau_balance.csv'
bureau = 'bureau.csv'
cc_bal = 'credit_card_balance.csv'
installments = 'installments_payments.csv'
POS_CASH = 'POS_CASH_balance.csv'
prev_app = 'previous_application.csv'
# create and view test df
test_uri = bucket + '/' + test
test = pd.read_csv(test_uri)
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
bureau_bal_uri = bucket + '/' + bureau_bal
bureau_bal = pd.read_csv(bureau_bal_uri)
bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [9]:
# create and view bureau df
bureau_uri = bucket + '/' + bureau
bureau = pd.read_csv(bureau_uri)
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [10]:
## CC Balance data
cc_bal_uri = bucket + '/' + cc_bal
credit_card_balance = pd.read_csv(cc_bal_uri)
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [11]:
# create and view installments df
installments_uri = bucket + '/' + installments
installments_payments = pd.read_csv(installments_uri)
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [12]:
# create and view POS_cash df
POS_CASH_uri = bucket + '/' + POS_CASH
POS_CASH_balance = pd.read_csv(POS_CASH_uri)
POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [13]:
# create and view prev_app df
prev_app_uri = bucket + '/' + prev_app
previous_application = pd.read_csv(prev_app_uri)
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [15]:
# Missing values statistics
missing_values = missing_values_table(test)
missing_values.head()

Your selected dataframe has 121 columns.
There are 64 columns that have missing values.


,Missing Values,% of Total Values
COMMONAREA_MODE,33495,68.7
COMMONAREA_MEDI,33495,68.7
COMMONAREA_AVG,33495,68.7
NONLIVINGAPARTMENTS_MEDI,33347,68.4
NONLIVINGAPARTMENTS_AVG,33347,68.4


In [16]:
## Exclude categorical data from the customer past data
mask = previous_application.dtypes != 'category'
# column = previous_application.columns
# mask
# column[mask==True]
previous_application.select_dtypes(exclude=['category']).columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

In [17]:
# previous_application
prev_app_group = previous_application.groupby('SK_ID_CURR').agg({'SK_ID_CURR':'count','AMT_ANNUITY':['sum','mean'],
'AMT_APPLICATION':['sum','mean'],'AMT_CREDIT':['sum','mean'],
'AMT_DOWN_PAYMENT':['sum','mean'],'AMT_GOODS_PRICE':['sum','mean'],
'NFLAG_LAST_APPL_IN_DAY':'mean','RATE_DOWN_PAYMENT':'mean','RATE_INTEREST_PRIMARY':'mean','RATE_INTEREST_PRIVILEGED':'mean',
'DAYS_DECISION':['sum','mean'],'SELLERPLACE_AREA':['sum','mean'],
'CNT_PAYMENT':['sum','mean'],'DAYS_FIRST_DRAWING':'mean','DAYS_FIRST_DUE':'mean',
'DAYS_LAST_DUE_1ST_VERSION':'mean','DAYS_LAST_DUE':'mean','DAYS_TERMINATION':'mean','NFLAG_INSURED_ON_APPROVAL':'mean'})
prev_app_group.columns = [' '.join(col).strip() for col in prev_app_group.columns.values]

In [18]:
prev_app_group.reset_index().head()

,SK_ID_CURR,SK_ID_CURR count,AMT_ANNUITY sum,AMT_ANNUITY mean,AMT_APPLICATION sum,AMT_APPLICATION mean,AMT_CREDIT sum,AMT_CREDIT mean,AMT_DOWN_PAYMENT sum,AMT_DOWN_PAYMENT mean,...,SELLERPLACE_AREA sum,SELLERPLACE_AREA mean,CNT_PAYMENT sum,CNT_PAYMENT mean,DAYS_FIRST_DRAWING mean,DAYS_FIRST_DUE mean,DAYS_LAST_DUE_1ST_VERSION mean,DAYS_LAST_DUE mean,DAYS_TERMINATION mean,NFLAG_INSURED_ON_APPROVAL mean
0,100001,1,3951.000,3951.000,24835.5,24835.50,23787.0,23787.00,2520.0,2520.0,...,23,23.0,8.0,8.0,365243.0,-1709.000000,-1499.000000,-1619.000000,-1612.000000,0.000000
1,100002,1,9251.775,9251.775,179055.0,179055.00,179055.0,179055.00,0.0,0.0,...,500,500.0,24.0,24.0,365243.0,-565.000000,125.000000,-25.000000,-17.000000,0.000000
2,100003,3,169661.970,56553.990,1306309.5,435436.50,1452573.0,484191.00,6885.0,3442.5,...,1599,533.0,30.0,10.0,365243.0,-1274.333333,-1004.333333,-1054.333333,-1047.333333,0.666667
3,100004,1,5357.250,5357.250,24282.0,24282.00,20106.0,20106.00,4860.0,4860.0,...,30,30.0,4.0,4.0,365243.0,-784.000000,-694.000000,-724.000000,-714.000000,0.000000
4,100005,2,4813.200,4813.200,44617.5,22308.75,40153.5,20076.75,4464.0,4464.0,...,36,18.0,12.0,12.0,365243.0,-706.000000,-376.000000,-466.000000,-460.000000,0.000000


In [19]:
# installments_payments
installments_payments_numeric = installments_payments.select_dtypes(exclude=['category'])
installments_payments_numeric['DAYS_ENTRY_PAYMENT'].fillna(0,inplace = True)
installments_payments_numeric['AMT_PAYMENT'].fillna(0,inplace = True)
installments_payments_numeric['late_pay_days'] = installments_payments_numeric['DAYS_INSTALMENT'] - installments_payments_numeric['DAYS_ENTRY_PAYMENT']
installments_payments_numeric['less_pay_amount'] = installments_payments_numeric['AMT_PAYMENT'] - installments_payments_numeric['AMT_INSTALMENT']
installments_payments_numeric

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,late_pay_days,less_pay_amount
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,7.0,0.000
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0.0,0.000
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0.0,0.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,8.0,0.000
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,-17.0,-4.455
...,...,...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,0.0,67.500,0.000,-1624.0,-67.500
13605397,1310347,414406,0.0,47,-1539.0,0.0,67.500,0.000,-1539.0,-67.500
13605398,1308766,402199,0.0,43,-7.0,0.0,43737.435,0.000,-7.0,-43737.435
13605399,1062206,409297,0.0,43,-1986.0,0.0,67.500,0.000,-1986.0,-67.500


In [20]:
inst_pay_group = installments_payments_numeric.groupby('SK_ID_CURR').agg({'SK_ID_CURR':'count','late_pay_days':['mean','sum'],'less_pay_amount':['mean','sum']})
inst_pay_group.columns = [' '.join(col).strip() for col in inst_pay_group.columns.values]

In [21]:
inst_pay_group.reset_index().head()

,SK_ID_CURR,SK_ID_CURR count,late_pay_days mean,late_pay_days sum,less_pay_amount mean,less_pay_amount sum
0,100001,7,7.285714,51.0,0.0,0.0
1,100002,19,20.421053,388.0,0.0,0.0
2,100003,25,7.160000,179.0,0.0,0.0
3,100004,3,7.666667,23.0,0.0,0.0
4,100005,9,23.555556,212.0,0.0,0.0


In [22]:
# bureau
bureau_numeric = bureau.select_dtypes(exclude=['category'])
bureau_numeric.isnull().mean()
bureau_numeric = bureau_numeric.drop(['AMT_ANNUITY','AMT_CREDIT_MAX_OVERDUE'],axis = 1)
bureau_numeric['AMT_CREDIT_SUM_DEBT'].fillna(0,inplace = True)
bureau_numeric['AMT_CREDIT_SUM'].fillna(0,inplace = True)
bureau_numeric['balance'] = bureau_numeric['AMT_CREDIT_SUM'] - bureau_numeric['AMT_CREDIT_SUM_DEBT']
bureau_numeric.isnull().mean()

SK_ID_CURR                0.000000
SK_ID_BUREAU              0.000000
CREDIT_ACTIVE             0.000000
CREDIT_CURRENCY           0.000000
DAYS_CREDIT               0.000000
CREDIT_DAY_OVERDUE        0.000000
DAYS_CREDIT_ENDDATE       0.061496
DAYS_ENDDATE_FACT         0.369170
CNT_CREDIT_PROLONG        0.000000
AMT_CREDIT_SUM            0.000000
AMT_CREDIT_SUM_DEBT       0.000000
AMT_CREDIT_SUM_LIMIT      0.344774
AMT_CREDIT_SUM_OVERDUE    0.000000
CREDIT_TYPE               0.000000
DAYS_CREDIT_UPDATE        0.000000
balance                   0.000000
dtype: float64

In [23]:
bureau_group = bureau_numeric.groupby('SK_ID_CURR').agg({'SK_ID_CURR':'count','DAYS_CREDIT':'mean','CREDIT_DAY_OVERDUE':'mean',
'DAYS_CREDIT_ENDDATE':'mean','DAYS_ENDDATE_FACT':'mean','CNT_CREDIT_PROLONG':['mean','sum'],'AMT_CREDIT_SUM_LIMIT':'mean',
'AMT_CREDIT_SUM_OVERDUE':['mean','sum'],'DAYS_CREDIT_UPDATE':'mean','balance':['mean','sum']})
bureau_group.columns = [' '.join(col).strip() for col in bureau_group.columns.values]
bureau_group

,SK_ID_CURR count,DAYS_CREDIT mean,CREDIT_DAY_OVERDUE mean,DAYS_CREDIT_ENDDATE mean,DAYS_ENDDATE_FACT mean,CNT_CREDIT_PROLONG mean,CNT_CREDIT_PROLONG sum,AMT_CREDIT_SUM_LIMIT mean,AMT_CREDIT_SUM_OVERDUE mean,AMT_CREDIT_SUM_OVERDUE sum,DAYS_CREDIT_UPDATE mean,balance mean,balance sum
SK_ID_CURR,,,,,,,,,,,,,
100001,7,-735.000000,0.0,82.428571,-825.500000,0.000000,0,0.00000,0.0,0.0,-93.142857,122382.642857,856678.500
100002,8,-874.000000,0.0,-349.000000,-697.500000,0.000000,0,7997.14125,0.0,0.0,-499.875000,77409.320625,619274.565
100003,4,-1400.750000,0.0,-544.500000,-1097.333333,0.000000,0,202500.00000,0.0,0.0,-816.000000,254350.125000,1017400.500
100004,2,-867.000000,0.0,-488.500000,-532.500000,0.000000,0,0.00000,0.0,0.0,-532.000000,94518.900000,189037.800
100005,3,-190.666667,0.0,439.333333,-123.000000,0.000000,0,0.00000,0.0,0.0,-54.333333,29572.500000,88717.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,13,-1667.076923,0.0,-1232.333333,-1364.750000,0.000000,0,0.00000,0.0,0.0,-1064.538462,271599.050769,3530787.660
456250,3,-862.000000,0.0,1288.333333,-760.000000,0.000000,0,19422.79500,0.0,0.0,-60.333333,284806.485000,854419.455
456253,4,-867.500000,0.0,280.500000,-794.000000,0.000000,0,0.00000,0.0,0.0,-253.250000,541041.750000,2164167.000


In [24]:
bureau_group.reset_index().head()

,SK_ID_CURR,SK_ID_CURR count,DAYS_CREDIT mean,CREDIT_DAY_OVERDUE mean,DAYS_CREDIT_ENDDATE mean,DAYS_ENDDATE_FACT mean,CNT_CREDIT_PROLONG mean,CNT_CREDIT_PROLONG sum,AMT_CREDIT_SUM_LIMIT mean,AMT_CREDIT_SUM_OVERDUE mean,AMT_CREDIT_SUM_OVERDUE sum,DAYS_CREDIT_UPDATE mean,balance mean,balance sum
0,100001,7,-735.000000,0.0,82.428571,-825.500000,0.0,0,0.00000,0.0,0.0,-93.142857,122382.642857,856678.500
1,100002,8,-874.000000,0.0,-349.000000,-697.500000,0.0,0,7997.14125,0.0,0.0,-499.875000,77409.320625,619274.565
2,100003,4,-1400.750000,0.0,-544.500000,-1097.333333,0.0,0,202500.00000,0.0,0.0,-816.000000,254350.125000,1017400.500
3,100004,2,-867.000000,0.0,-488.500000,-532.500000,0.0,0,0.00000,0.0,0.0,-532.000000,94518.900000,189037.800
4,100005,3,-190.666667,0.0,439.333333,-123.000000,0.0,0,0.00000,0.0,0.0,-54.333333,29572.500000,88717.500


In [25]:
# POS_CASH_balanc
POS_CASH = POS_CASH_balance.select_dtypes(exclude=['category'])
POS_CASH.isnull().mean()
POS_CASH.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [26]:
POS_CASH_group = POS_CASH.groupby('SK_ID_CURR').agg({'SK_ID_CURR':'count','MONTHS_BALANCE':'mean','CNT_INSTALMENT':'mean',
'CNT_INSTALMENT_FUTURE':['mean','sum'],'SK_DPD':['mean','sum'],'SK_DPD_DEF':['mean','sum']})
POS_CASH_group.columns = [' '.join(col).strip() for col in POS_CASH_group.columns.values]
POS_CASH_group

,SK_ID_CURR count,MONTHS_BALANCE mean,CNT_INSTALMENT mean,CNT_INSTALMENT_FUTURE mean,CNT_INSTALMENT_FUTURE sum,SK_DPD mean,SK_DPD sum,SK_DPD_DEF mean,SK_DPD_DEF sum
SK_ID_CURR,,,,,,,,,
100001,9,-72.555556,4.000000,1.444444,13.0,0.777778,7,0.777778,7
100002,19,-10.000000,24.000000,15.000000,285.0,0.000000,0,0.000000,0
100003,28,-43.785714,10.107143,5.785714,162.0,0.000000,0,0.000000,0
100004,4,-25.500000,3.750000,2.250000,9.0,0.000000,0,0.000000,0
100005,11,-20.000000,11.700000,7.200000,72.0,0.000000,0,0.000000,0
...,...,...,...,...,...,...,...,...,...
456251,9,-5.000000,7.875000,4.375000,35.0,0.000000,0,0.000000,0
456252,7,-79.000000,6.000000,3.000000,21.0,0.000000,0,0.000000,0
456253,17,-79.235294,6.705882,2.000000,34.0,0.294118,5,0.294118,5


In [27]:
POS_CASH_group.reset_index().head()

,SK_ID_CURR,SK_ID_CURR count,MONTHS_BALANCE mean,CNT_INSTALMENT mean,CNT_INSTALMENT_FUTURE mean,CNT_INSTALMENT_FUTURE sum,SK_DPD mean,SK_DPD sum,SK_DPD_DEF mean,SK_DPD_DEF sum
0,100001,9,-72.555556,4.000000,1.444444,13.0,0.777778,7,0.777778,7
1,100002,19,-10.000000,24.000000,15.000000,285.0,0.000000,0,0.000000,0
2,100003,28,-43.785714,10.107143,5.785714,162.0,0.000000,0,0.000000,0
3,100004,4,-25.500000,3.750000,2.250000,9.0,0.000000,0,0.000000,0
4,100005,11,-20.000000,11.700000,7.200000,72.0,0.000000,0,0.000000,0


In [28]:
# credit_card_balance
credit_numeric = credit_card_balance.select_dtypes(exclude=['category'])
credit_numeric.isnull().mean()
credit_numeric['AMT_DRAWINGS_ATM_CURRENT'].fillna(0,inplace = True)
credit_numeric['AMT_DRAWINGS_CURRENT'].fillna(0,inplace = True)
credit_numeric['AMT_DRAWINGS_OTHER_CURRENT'].fillna(0,inplace = True)
credit_numeric['AMT_DRAWINGS_POS_CURRENT'].fillna(0,inplace = True)
credit_numeric['amt_drawing'] = credit_numeric['AMT_DRAWINGS_ATM_CURRENT'] + credit_numeric['AMT_DRAWINGS_CURRENT'] + credit_numeric['AMT_DRAWINGS_OTHER_CURRENT'] + credit_numeric['AMT_DRAWINGS_POS_CURRENT']

credit_numeric['CNT_DRAWINGS_ATM_CURRENT'].fillna(0,inplace = True)
credit_numeric['CNT_DRAWINGS_CURRENT'].fillna(0,inplace = True)
credit_numeric['CNT_DRAWINGS_OTHER_CURRENT'].fillna(0,inplace = True)
credit_numeric['CNT_DRAWINGS_POS_CURRENT'].fillna(0,inplace = True)

credit_numeric['cnt_drawing'] = credit_numeric['CNT_DRAWINGS_ATM_CURRENT'] + credit_numeric['CNT_DRAWINGS_CURRENT'] + credit_numeric['CNT_DRAWINGS_OTHER_CURRENT'] + credit_numeric['CNT_DRAWINGS_POS_CURRENT']

credit_numeric = credit_numeric.drop(['CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_POS_CURRENT'],axis = 1)
credit_numeric

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,amt_drawing,cnt_drawing
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,1800.000,0.000,0.000,0.000,35.0,Active,0,0,1755.0,2.0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,2250.000,60175.080,64875.555,64875.555,69.0,Active,0,0,4500.0,2.0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,2250.000,26926.425,31460.085,31460.085,30.0,Active,0,0,0.0,0.0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,11925.000,224949.285,233048.970,233048.970,10.0,Active,0,0,4500.0,2.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,27000.000,443044.395,453919.455,453919.455,101.0,Active,0,0,23094.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.000,0.000,0.0,Active,0,0,0.0,0.0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.000,0.000,23.0,Active,0,0,0.0,0.0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,356994.675,269356.140,273093.975,273093.975,18.0,Active,0,0,540000.0,4.0
3840310,1624872,430337,-10,0.000,450000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.000,0.000,0.0,Active,0,0,0.0,0.0


In [29]:
credit_group = credit_numeric.groupby('SK_ID_CURR').agg({'SK_ID_CURR':'count','MONTHS_BALANCE':'mean',
'AMT_BALANCE':['mean','sum'],'AMT_CREDIT_LIMIT_ACTUAL':'mean','AMT_INST_MIN_REGULARITY':'mean',
'AMT_PAYMENT_TOTAL_CURRENT':['mean','sum'],'AMT_RECEIVABLE_PRINCIPAL':['mean','sum'],
'AMT_TOTAL_RECEIVABLE':['mean','sum'],'CNT_INSTALMENT_MATURE_CUM':['mean','sum'],
'SK_DPD':'mean', 'SK_DPD_DEF': 'mean',
'amt_drawing':['mean','sum'],'cnt_drawing':['mean','sum']})
credit_group.columns = [' '.join(col).strip() for col in credit_group.columns.values]
credit_group.columns

Index(['SK_ID_CURR count', 'MONTHS_BALANCE mean', 'AMT_BALANCE mean',
       'AMT_BALANCE sum', 'AMT_CREDIT_LIMIT_ACTUAL mean',
       'AMT_INST_MIN_REGULARITY mean', 'AMT_PAYMENT_TOTAL_CURRENT mean',
       'AMT_PAYMENT_TOTAL_CURRENT sum', 'AMT_RECEIVABLE_PRINCIPAL mean',
       'AMT_RECEIVABLE_PRINCIPAL sum', 'AMT_TOTAL_RECEIVABLE mean',
       'AMT_TOTAL_RECEIVABLE sum', 'CNT_INSTALMENT_MATURE_CUM mean',
       'CNT_INSTALMENT_MATURE_CUM sum', 'SK_DPD mean', 'SK_DPD_DEF mean',
       'amt_drawing mean', 'amt_drawing sum', 'cnt_drawing mean',
       'cnt_drawing sum'],
      dtype='object')

In [30]:
credit_group.reset_index().head()

,SK_ID_CURR,SK_ID_CURR count,MONTHS_BALANCE mean,AMT_BALANCE mean,AMT_BALANCE sum,AMT_CREDIT_LIMIT_ACTUAL mean,AMT_INST_MIN_REGULARITY mean,AMT_PAYMENT_TOTAL_CURRENT mean,AMT_PAYMENT_TOTAL_CURRENT sum,AMT_RECEIVABLE_PRINCIPAL mean,...,AMT_TOTAL_RECEIVABLE mean,AMT_TOTAL_RECEIVABLE sum,CNT_INSTALMENT_MATURE_CUM mean,CNT_INSTALMENT_MATURE_CUM sum,SK_DPD mean,SK_DPD_DEF mean,amt_drawing mean,amt_drawing sum,cnt_drawing mean,cnt_drawing sum
0,100006,6,-3.5,0.000000,0.000,270000.000000,0.000000,0.000000,0.000,0.000000,...,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,100011,74,-38.5,54482.111149,4031676.225,164189.189189,3956.221849,4520.067568,334485.000,52402.088919,...,54433.179122,4028055.255,25.767123,1881.0,0.000000,0.000000,4864.864865,360000.0,0.108108,8.0
2,100013,96,-48.5,18159.919219,1743352.245,131718.750000,1454.539551,6817.172344,654448.545,17255.559844,...,18101.079844,1737703.665,18.719101,1666.0,0.010417,0.010417,11906.250000,1143000.0,0.479167,46.0
3,100021,17,-10.0,0.000000,0.000,675000.000000,0.000000,0.000000,0.000,0.000000,...,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,100023,8,-7.5,0.000000,0.000,135000.000000,0.000000,0.000000,0.000,0.000000,...,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [72]:
test.shape

(48744, 121)

In [53]:
## Read data in real time from S3
def application_api(file1):
    applicant_details = file1.sample(10)
    return applicant_details


In [54]:
test_sample = application_api(test)

In [55]:
test_sample

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
17681,228169,0,0,0,1,0,157500.0,238500.0,23125.5,238500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
9604,169810,0,0,0,1,0,126000.0,773568.0,25083.0,612000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
30937,324788,0,0,0,1,1,135000.0,225000.0,17775.0,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
20363,247852,0,1,1,1,0,675000.0,1773000.0,55161.0,1773000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0
9610,169864,0,1,0,1,0,315000.0,276277.5,21955.5,238500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
32732,338426,0,0,0,0,0,225000.0,904500.0,32485.5,904500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
17889,229631,0,1,0,1,0,112500.0,298728.0,15381.0,202500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
48468,454085,0,1,1,1,0,180000.0,550980.0,43659.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
40657,399186,0,0,0,0,0,135000.0,152820.0,13068.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0
30063,318763,0,1,1,1,0,270000.0,553581.0,35509.5,472500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [73]:
## Merge test data with other customer data
test_merge = test_sample.merge(prev_app_group, on='SK_ID_CURR', how='left')
test_merge = test_merge.merge(bureau_group, on='SK_ID_CURR', how='left')
test_merge = test_merge.merge(credit_group, on='SK_ID_CURR', how='left')
test_merge = test_merge.merge(POS_CASH_group, on='SK_ID_CURR', how='left')
test_merge = test_merge.merge(inst_pay_group, on='SK_ID_CURR', how='left')

In [74]:
test_merge.shape

(10, 195)

In [75]:
def label_encoder(input_df, encoder_dict=None):
    """ Process a dataframe into a form useable by LightGBM """
    # Label encode categoricals
    categorical_feats = input_df.columns[input_df.dtypes == 'object']
    for feat in categorical_feats:
        encoder = LabelEncoder()
        input_df[feat] = encoder.fit_transform(input_df[feat].fillna('NULL'))
    return input_df, categorical_feats.tolist()

In [76]:

test_label, categorical_feats_test = label_encoder(test_merge)

In [77]:
test_label.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CNT_INSTALMENT_FUTURE sum,SK_DPD mean_y,SK_DPD sum,SK_DPD_DEF mean_y,SK_DPD_DEF sum,SK_ID_CURR count,late_pay_days mean,late_pay_days sum,less_pay_amount mean,less_pay_amount sum
0,228169,0,0,0,1,0,157500.0,238500.0,23125.5,238500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,169810,0,0,0,1,0,126000.0,773568.0,25083.0,612000.0,...,86.0,0.000000,0.0,0.000000,0.0,10.0,35.800000,358.0,0.000000,0.00
2,324788,0,0,0,1,1,135000.0,225000.0,17775.0,225000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,247852,0,1,1,1,0,675000.0,1773000.0,55161.0,1773000.0,...,20.0,0.000000,0.0,0.000000,0.0,5.0,10.400000,52.0,0.000000,0.00
4,169864,0,1,0,1,0,315000.0,276277.5,21955.5,238500.0,...,78.0,2.714286,38.0,2.714286,38.0,19.0,-0.473684,-9.0,-2682.298421,-50963.67


In [78]:
test_set= test_label.drop(['SK_ID_CURR'], axis=1)
# test_set = test_set.drop(["SK_ID_CURR count_x",'SK_ID_CURR count_y'],axis = 1)

In [79]:
test_set.shape

(10, 194)

In [80]:
test_set = test_set.drop(["SK_ID_CURR count_x",'SK_ID_CURR count_y'],axis = 1)

In [81]:
test_set.shape

(10, 190)

In [82]:
result = mdl_lightgbm.predict(test_set)

In [83]:
result

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0])

In [84]:
test_label['Decision'] = result

In [85]:
test_label['Decision']

0    1
1    0
2    0
3    0
4    1
5    0
6    0
7    0
8    1
9    0
Name: Decision, dtype: int64

In [86]:
## Store the predicted outcomes in S3
test_label.to_csv('s3://msba6330trendsmarketplace/result.csv')